In [ ]:
"""
okay, tasks for this notebook:

-make a teacher with descent class distribution
    --make a student of same architecture, train it (i have all these parts. ez.)
        ---using ints as input (default) and post-sigmoid as output
        ---remove last layer and use logits, then compare with sigmoide added on
        ---both?
    --swap the embedding layer for something that can take random noise.  add that to noiseKD? --on another nb
        ---this should enable us to train on noise, not just int indexes.  compare all
        
"""

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from NoiseKD import Teacher, SimpleLanguageModel, slm_init_config, slm_model_config, count_parameters

In [7]:
SLM = SimpleLanguageModel(**slm_init_config)

In [8]:
count_parameters(SLM)

3105922

In [9]:
slm_init_config['vocab_size']

80

In [10]:
teacher_slm = Teacher(SLM,(160,))

In [11]:
# for comparison, i need to be starting them off with the same init.  do that.

In [12]:
teacher_slm.configure(**slm_model_config)

Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)
lets try ints!


Configuring Teacher:: 100%|█████████████████████| 50/50 [01:36<00:00,  1.93s/it]

Teacher Configured


In [13]:
args = { 'val_train' : "train"
                      , 'n' : 100_000
                      , 'dist_type' : 'ints'
                      , 'm' : slm_init_config['vocab_size']
                      , 'std': 1.0
        }
teacher_slm.generate_data(**args)

Generating train data :: 100%|██████████████| 2000/2000 [00:51<00:00, 39.10it/s]


In [14]:
args = { 'val_train' : "val"
                      , 'n' : 10_000
                      , 'dist_type' : 'ints'
                      , 'm' : slm_init_config['vocab_size']
                      , 'std': 1.0
        }
teacher_slm.generate_data(**args)

Generating val data :: 100%|██████████████████| 200/200 [00:04<00:00, 40.07it/s]


In [15]:
student = SimpleLanguageModel(**slm_init_config)

In [16]:
student

SimpleLanguageModel(
  (embedding): Embedding(80, 16)
  (transformer_encoder): TransformerEncoder(
    (transformer): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
          )
          (linear1): Linear(in_features=16, out_features=11, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=11, out_features=16, bias=True)
          (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
       

In [ ]:
##to train form the teacher object was written already

In [26]:
num_epochs = 10
batch_size = 32
learning_rate = 0.009 #0.00025 this shit is slooooow
momentum = 0.95
criterion = nn.MSELoss()
optimizer = optim.Adam(student.parameters(), lr=learning_rate)

optimizer = optim.SGD(student.parameters(), lr=learning_rate, momentum=momentum)
data = list(zip(teacher_slm.train_inputs, teacher_slm.train_targets))
input_tensors = torch.stack([torch.Tensor(x[0]) for x in data])
target_tensors = torch.stack([torch.Tensor(x[1]) for x in data])
dataset = TensorDataset(input_tensors, target_tensors)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

val_data = list(zip(teacher_slm.val_inputs, teacher_slm.val_targets))
val_input_tensors = torch.stack([torch.Tensor(x[0]) for x in val_data])
val_target_tensors = torch.stack([torch.Tensor(x[1]) for x in val_data])
val_dataset = TensorDataset(val_input_tensors, val_target_tensors)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [27]:
#conventional train, repeats throught the dataset
accuracy_threshold = 0.5
print_every = 1 #its working, its just lots of data mama.
validation_every = 1
losses = []  # List to store losses
accuracies = []

for epoch in range(num_epochs):
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    for input_batch, target_batch in dataloader:
        optimizer.zero_grad()  # Zero the gradients
        output = student(input_batch)  # Forward pass
        loss = criterion(output, target_batch)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the weights
        total_loss += loss.item()
        #add early stopping...
        #and validation at each step.  
        # Calculate accuracy
        predictions = (output > accuracy_threshold).float()  # Assuming a threshold of 0.5 for binary classification
        correct_predictions += (predictions == target_batch).sum().item()
        total_samples += input_batch.size(0)

    # Print the average loss for this epoch
    avg_loss = total_loss / len(dataloader)
    losses.append(avg_loss)
        
    accuracy = correct_predictions / total_samples
    accuracies.append(accuracy)

    if (epoch + 1) % print_every == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.8f}, Train Accuracy: {accuracy:.4f}')

    if (epoch + 1) % validation_every == 0:
        student.eval()
        
        total_val_samples = 0
        correct_val_predictions = 0

        with torch.no_grad():
            for val_input_batch, val_target_batch in val_dataloader:
                val_output = student(val_input_batch)
                val_predictions = (val_output > accuracy_threshold).float()  # Assuming a threshold of 0.5 for binary classification
                correct_val_predictions += (val_predictions == val_target_batch).sum().item()
                total_val_samples += val_input_batch.size(0)

        # Calculate validation accuracy
        val_accuracy = correct_val_predictions / total_val_samples

        # Print the validation accuracy for this epoch
        print(f'\t\tValidation Accuracy: {val_accuracy:.4f}')

        # Set the model back to training mode
        student.train()
            
            

Epoch [1/10], Loss: 0.00728246, Train Accuracy: 79.1964
		Validation Accuracy: 79.1895
Epoch [2/10], Loss: 0.00726915, Train Accuracy: 79.1964
		Validation Accuracy: 79.1894
Epoch [3/10], Loss: 0.00725322, Train Accuracy: 79.1967
		Validation Accuracy: 79.1894
Epoch [4/10], Loss: 0.00723599, Train Accuracy: 79.1971
		Validation Accuracy: 79.1873
Epoch [5/10], Loss: 0.00721645, Train Accuracy: 79.1979
		Validation Accuracy: 79.1894
Epoch [6/10], Loss: 0.00719714, Train Accuracy: 79.1985
		Validation Accuracy: 79.1869
Epoch [7/10], Loss: 0.00718090, Train Accuracy: 79.2000
		Validation Accuracy: 79.1857
Epoch [8/10], Loss: 0.00716581, Train Accuracy: 79.2015
		Validation Accuracy: 79.1846
Epoch [9/10], Loss: 0.00714748, Train Accuracy: 79.2035
		Validation Accuracy: 79.1806
Epoch [10/10], Loss: 0.00712604, Train Accuracy: 79.2053
		Validation Accuracy: 79.1887


In [ ]:
#all unique train, 

##expand this then try collab
#scc? or na? that competes with the other stuff i bet

print_every = 1
batch_eternal = 50
data_n = 5000
num_epochs = 400
losses = []  # List to store losses

for epoch in range(num_epochs):
    total_loss = 0
    teacher.generate_data(
    "train"
    ,data_n
    ,'normal'
    , m =0.0
    , std=1.0
    )
    data = list(zip(teacher.train_inputs, teacher.train_targets))
    input_tensors = torch.stack([torch.Tensor(x[0]) for x in data])
    target_tensors = torch.stack([torch.Tensor(x[1]) for x in data])
    dataset = TensorDataset(input_tensors, target_tensors)
    dataloader = DataLoader(dataset, batch_size=batch_eternal, shuffle=True)
    
    for input_batch, target_batch in dataloader:
        optimizer.zero_grad()  # Zero the gradients
        output = student(input_batch)  # Forward pass
        loss = criterion(output, target_batch)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the weights
        total_loss += loss.item()

    # Print the average loss for this epoch
    avg_loss = total_loss / len(dataloader)
    losses.append(avg_loss)
    if (epoch + 1) % print_every == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.8f}')